# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip install --upgrade tensorflow keras Pillow

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [3]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:

tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cnn/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cnn/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [48]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [49]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [50]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [51]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [52]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [53]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [54]:
cnn.add(tf.keras.layers.Dense(units=128, activation='sigmoid'))

### Step 5 - Output Layer

In [55]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [56]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [57]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 56s 221ms/step - loss: 0.6902 - accuracy: 0.5369 - val_loss: 0.6417 - val_accuracy: 0.6375
Epoch 2/25
250/250 [==============================] - 54s 218ms/step - loss: 0.6436 - accuracy: 0.6265 - val_loss: 0.6193 - val_accuracy: 0.6545
Epoch 3/25
250/250 [==============================] - 55s 218ms/step - loss: 0.5980 - accuracy: 0.6873 - val_loss: 0.5677 - val_accuracy: 0.7215
Epoch 4/25
250/250 [==============================] - 54s 217ms/step - loss: 0.5639 - accuracy: 0.7090 - val_loss: 0.5401 - val_accuracy: 0.7285
Epoch 5/25
250/250 [==============================] - 54s 218ms/step - loss: 0.5334 - accuracy: 0.7295 - val_loss: 0.4991 - val_accuracy: 0.7565
Epoch 6/25
250/250 [==============================] - 54s 218ms/step - loss: 0.5081 - accuracy: 0.7498 - val_loss: 0.4915 - val_accuracy: 0.7640
Epoch 7/25
250/250 [==============================] - 54s 216ms/step - loss: 0.4774 - accuracy: 0.7690 - val_loss: 0.5022 - val_ac

In [17]:
import torch

In [18]:
def testar_gpu():
	train_on_gpu = torch.cuda.is_available() #Observa se a GPU está disponivel
	if train_on_gpu: #Se sim
		device = torch.device('cuda') #Seleciona o device como GPU
		print("Treinando na GPU") #E manda a mensagem
	else: #Se não
		device = torch.device('cpu') #Seleciona o device como cpu
		print("GPU indisponível, treinando na CPU") #E avisa que a GPU não esta disponível
	return device

device = testar_gpu()

Treinando na GPU


## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)

dog


## Fim